In [4]:
import urllib
import re
from bs4 import BeautifulSoup
import time
import os.path
from os import path
import lxml

In [10]:
#genre_list=["sculpture","portrait","nude painting (nu)"]
#genre_list=["portrait", "sculpture", "nude painting (nu)"]
genre_list=["sculpture"]

In [9]:
movement = "Hellenistic Period (323-31 BC)"

In [7]:
movement_list = ["Hellenistic Period (323-31 BC)",
"Gothic Art",
"Coptic art",
"Proto Renaissance ",
"Early Renaissance",
"High Renaissance ",
"Mannerism (Late Renaissance)",
"Northern Renaissance",
"Baroque",
"Dutch Golden Age",
"Rococo",
"Neoclassicism",
"Romanticism",
"Orientalism",
"Academic Art",
"Realism"]

In [12]:
for genre in genre_list:
    file_path = 'C:/Users/WS2/Desktop/kozmiksefer/disruption/data_set/'+str(genre)+'_'+str(movement)
    print(file_path)
    if path.exists(file_path) == False:
      os.makedirs(file_path)
      os.chdir(file_path)

C:/Users/WS2/Desktop/kozmiksefer/disruption/data_set/sculpture_Hellenistic Period (323-31 BC)


In [13]:
base_url = "https://www.wikiart.org"
# iterate through all artists by last name alphabetically
# like this https://www.wikiart.org/en/Alphabet/m/text-list
for c in range(ord('a'), ord('z') + 1):
  char = chr(c)
  artist_list_url = base_url + "/en/Alphabet/" + char + "/text-list"
  print(artist_list_url)

  genre_soup = BeautifulSoup(urllib.request.urlopen(artist_list_url), "lxml")
  artist_list_main = genre_soup.find("main")
  lis = artist_list_main.find_all("li")

  # for each list element
  for li in lis: 
    born = 0
    died = 0

    # get the date range
    for line in li.text.splitlines():
      if line.startswith(",") and "-" in line:
        parts = line.split('-')
        if len(parts) == 2:
          born = int(re.sub("[^0-9]", "",parts[0]))
          died = int(re.sub("[^0-9]", "",parts[1]))

    link = li.find("a")
    artist = link.attrs["href"]

    # get the artist's main page
    artist_url = base_url + artist
    artist_soup = BeautifulSoup(urllib.request.urlopen(artist_url), "lxml")

    # only look for artists with the word abstract on their main page
    for genre in genre_list:
      if genre in artist_soup.text and movement in artist_soup.text: 
        print(artist + " " + str(born) + " - " + str(died))
        #artist_soup.text

        # get the artist's web page for the artwork
        url = base_url + artist + '/all-works/text-list'
        artist_work_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

        # get the main section
        artist_main = artist_work_soup.find("main")
        image_count = 0
        artist_name = artist.split("/")[2]
        # get the list of artwork
        lis = artist_main.find_all("li")

        # for each list element
        for li in lis:
          link = li.find("a")

          if link != None:
            painting = link.attrs["href"]

            # get the painting
            url = base_url + painting
            print(url)

            try:
              painting_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

            except:
              print("error retreiving page")
              continue

            #check the genre
            genre_field = painting_soup.find("span", {"itemprop":"genre"})
            print("genre:",genre_field, "bizim aradığımıız:",genre)
            if genre_field != None and genre in genre_field:
              print("field :", genre_field)

              # get the url
              og_image = painting_soup.find("meta", {"property":"og:image"})
              image_url = og_image["content"].split("!")[0] # ignore the !Large.jpg at the end
              print(image_url)

              parts = url.split("/")
              painting_name = parts[-1]
              file_path='C:/Users/WS2/Desktop/kozmiksefer/disruption/data_set/'+str(genre)
              save_path = file_path + "/" + artist_name + "_" + painting_name + ".jpg"

              #download the file
              try:
                print("downloading to " + save_path)
                time.sleep(0.2)  # try not to get a 403                    
                urllib.request.urlretrieve(image_url, save_path)
                image_count = image_count + 1
              except Exception as e:
                print("failed downloading " + image_url, e) 
                      

https://www.wikiart.org/en/Alphabet/a/text-list
/en/ancient-greek-painting 700 - 31
https://www.wikiart.org/en/ancient-greek-painting/terracotta-vase-in-the-form-of-a-ketos-sea-monster--650
genre: <span itemprop="genre">sculpture</span> bizim aradığımıız: sculpture
field : <span itemprop="genre">sculpture</span>
https://uploads4.wikiart.org/00237/images/ancient-greek-painting/terracotta-vase-in-the-form-of-a-ketos-sea-monster-650-1.jpg
downloading to C:/Users/WS2/Desktop/kozmiksefer/disruption/data_set/sculpture/ancient-greek-painting_terracotta-vase-in-the-form-of-a-ketos-sea-monster--650.jpg
https://www.wikiart.org/en/ancient-greek-painting/fragment-showing-perseus-with-the-head-of-medusa-likely-from-a-metope-from-the-temple-of-apollo-at--630
genre: None bizim aradığımıız: sculpture
https://www.wikiart.org/en/ancient-greek-painting/terracotta-head-of-a-man--600
genre: None bizim aradığımıız: sculpture
https://www.wikiart.org/en/ancient-greek-painting/a-color-reconstruction-of-a-lion-

KeyboardInterrupt: 